# －－－－主題 : 財報公布前預測企業價值,持有報酬高?－－－－－

## ．目錄:
一、 資料說明

二、 資料處理

三、 迴圈前測試

四、 迴圈---以code做groupby,做預測

## 一、【資料說明】
1. 資料樣本 : 台灣加權股市上市股票
2. 資料期間 : 2008~2017
3. 輸入變數 : 財報資訊  
（GPM、OPM、NIM、EBITDA、現金、應收帳款、存貨、固定資產、廠房與設備投資、流動資產、流動負債、leverage、revenue、市值、GOR）

In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, ensemble, preprocessing, metrics

．輸入資料　（pd.read_csv）

In [2]:
data=pd.read_csv("python_data1.csv",engine='python')

．查看前2筆資料　（.head(2)）

In [3]:
data.head(2)

,code,date,GPM,OPM,NIM,EBITDA,cash,account_receviable,inventory,fixed_asset,invest,liq_asset,liq_liability,leverage,revenue,mkt_value,GOR
0,1101,200803,13.91,9.89,9.53,4041764,12423117,7631637,7302870,115287797,-2650659,57091512,29783960,46.86,17906537,191344000.0,-0.1019
1,1101,200806,12.17,8.28,11.23,8364504,11760556,9501174,6841077,119771536,-7462419,54362283,42792997,51.40,38460834,133194000.0,0.0683


．觀察資料缺值、型態　（.info()）

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25960 entries, 0 to 25959
Data columns (total 17 columns):
code                  25960 non-null int64
date                  25960 non-null int64
GPM                   25960 non-null float64
OPM                   25960 non-null float64
NIM                   25960 non-null float64
EBITDA                25960 non-null int64
cash                  25960 non-null int64
account_receviable    25960 non-null int64
inventory             25960 non-null int64
fixed_asset           25960 non-null int64
invest                25960 non-null int64
liq_asset             25960 non-null int64
liq_liability         25960 non-null int64
leverage              25960 non-null float64
revenue               25960 non-null int64
mkt_value             25960 non-null float64
GOR                   25960 non-null float64
dtypes: float64(6), int64(11)
memory usage: 3.4 MB


筆記: 合併資料-->  類似R裡面的c-bind: pd.concat ( [A,B] ,axis=1 )    |  R裡面的r-bind: A.append(B)

## 二、【資料處理】

新增欄位:

In [5]:
# 企業價值(EV)欄位
data['EV'] = data['mkt_value'] + data['liq_liability'] - data['cash']

# 計算EBITDA/EV，並將數值X1000 (為了使後續轉整數值時，仍可保持數值相對大小。如:將0.019轉成19)
data['EBITDA_EV'] = (data['EBITDA'] / data['EV'])*1000   # 0.1% 單位

# 將如上述相同的GOR數值X100
data['GOR'] = data['GOR']*100

將數值轉整數:　（.astype(int)）

In [6]:
data = data.astype(int)

敘述統計:　　（.describe()）

In [7]:
data.describe()

,code,date,GPM,OPM,NIM,EBITDA,cash,account_receviable,inventory,fixed_asset,invest,liq_asset,liq_liability,leverage,revenue,mkt_value,GOR,EV,EBITDA_EV
count,25960.000000,25960.000000,25960.000000,25960.000000,25960.000000,2.596000e+04,2.596000e+04,2.596000e+04,2.596000e+04,2.596000e+04,2.596000e+04,2.596000e+04,2.596000e+04,25960.000000,2.596000e+04,2.596000e+04,25960.000000,2.596000e+04,25960.000000
mean,3366.784284,201257.500000,19.193413,-7.795185,1.429545,1.877395e+06,4.359460e+06,4.545351e+06,4.301350e+06,9.354451e+06,-8.061903e+05,1.553805e+07,1.076097e+07,42.571726,1.636215e+07,2.200790e+07,52.925270,2.784365e+07,57.502465
std,2351.025044,287.253248,18.725125,401.746241,678.636725,8.046860e+06,2.277929e+07,2.318394e+07,1.682119e+07,3.217948e+07,3.370938e+06,6.527966e+07,4.785069e+07,17.557707,7.519858e+07,8.074474e+07,2894.147994,9.971105e+07,100.636994
min,1101.000000,200803.000000,-557.000000,-32157.000000,-22185.000000,-2.146405e+07,4.840000e+02,0.000000e+00,0.000000e+00,0.000000e+00,-9.596002e+07,-1.548831e+09,1.475000e+03,0.000000,-1.916478e+09,8.500000e+04,-99.000000,-2.147484e+09,-3196.000000
25%,1733.000000,201008.250000,9.000000,0.000000,0.000000,8.800350e+04,3.387805e+05,3.124900e+05,3.721892e+05,6.244542e+05,-3.571130e+05,1.846876e+06,8.713890e+05,30.000000,1.101516e+06,2.214750e+06,-11.000000,2.938812e+06,21.000000
50%,2464.000000,201257.500000,17.000000,4.000000,4.000000,3.250075e+05,9.045880e+05,9.186610e+05,9.416095e+05,1.659830e+06,-8.008600e+04,4.094638e+06,2.221060e+06,43.000000,3.048604e+06,5.188500e+06,0.000000,6.818246e+06,48.000000
75%,3481.000000,201506.750000,27.000000,10.000000,9.000000,1.009981e+06,2.414978e+06,2.547179e+06,2.772695e+06,4.976073e+06,-1.530900e+04,9.887622e+06,6.163521e+06,55.000000,8.930965e+06,1.243225e+07,13.000000,1.636509e+07,88.000000
max,9958.000000,201712.000000,100.000000,160.000000,81135.000000,2.891660e+08,7.550959e+08,1.229237e+09,5.609549e+08,4.663030e+08,0.000000e+00,2.045529e+09,2.025502e+09,102.000000,2.045071e+09,2.027462e+09,417465.000000,2.078569e+09,5802.000000


偏態:  　　　（.skew(axis=0)）

In [8]:
data.skew(axis=0)

code                    1.575811
date                    0.000000
GPM                    -5.194147
OPM                   -54.849269
NIM                    77.035320
EBITDA                 14.442787
cash                   21.175347
account_receviable     20.561184
inventory              15.891328
fixed_asset             7.945374
invest                -11.958904
liq_asset              16.998355
liq_liability          18.068881
leverage                0.072254
revenue                 7.855768
mkt_value               9.680757
GOR                   120.939992
EV                      7.746659
EBITDA_EV              10.405664
dtype: float64

In [9]:
data.head()

,code,date,GPM,OPM,NIM,EBITDA,cash,account_receviable,inventory,fixed_asset,invest,liq_asset,liq_liability,leverage,revenue,mkt_value,GOR,EV,EBITDA_EV
0,1101,200803,13,9,9,4041764,12423117,7631637,7302870,115287797,-2650659,57091512,29783960,46,17906537,191344000,-10,208704843,19
1,1101,200806,12,8,11,8364504,11760556,9501174,6841077,119771536,-7462419,54362283,42792997,51,38460834,133194000,6,164226441,50
2,1101,200809,11,7,9,11993727,10615331,9283486,8550177,122733210,-10158525,45692556,39599056,52,60944205,61399000,10,90382725,132
3,1101,200812,9,6,8,15111414,13033535,8386187,6908626,125807120,-14729131,43762069,39174857,51,78476558,88724000,2,114865322,131
4,1101,200903,9,5,2,2829888,14200950,9431949,5539320,127230295,-1963180,48506002,38714520,51,18071221,92016000,0,116529570,24


## 三、【迴圈前測試】

建立 code 清單

In [10]:
code_list=pd.DataFrame(data['code'].unique())
code_list.columns=['code']

建立空資料（DataFrame）以記錄迴圈後資料

In [11]:
result = pd.DataFrame()    

－－－－－－－－－－－－－－－－－－－－－－　開始試迴圈內的程式　－－－－－－－－－－－－－－－－－－－－－－

### ．個股新資料-->pre_data　資料

選擇清單中第2列,第1欄資料

In [26]:
code_list.iloc[1,0]

1102

依照個股代號，從母體資料切割成新資料 (這裡命名為　pre_data　資料)

In [27]:
# pre_data
pre_data = data[data['code']==code_list.iloc[1,0]]

將 EBITDA_EV 欄位做lag一期。  （.shift(+1)）

再將 EBITDA_EV 與前一季做相減，作為　change　欄位的資料。 

且將第一筆缺值去除。　（.dropna()）

In [ ]:
pre_data['EBITDA_EV_lag'] = pre_data['EBITDA_EV'].shift(+1)
pre_data['change'] = pre_data['EBITDA_EV']-pre_data['EBITDA_EV_lag']
pre_data = pre_data.dropna()

若 EBITDA/EV 欄位比上一季，變動>3%。

則 label 欄位為 1 ,否則為 0          （　np.where(data['A']>=30,1,0) 類似R的IF條件句　）

In [29]:
pre_data['label'] = np.where(pre_data['change']>=30,1,0)

### ．train資料 、text資料

201512 前 為訓練集資料

201512 後 為測試集資料

In [30]:
train = pre_data[pre_data['date']<=201512]
test = pre_data[pre_data['date']>201512]
#text = text[text['date']<=201612]

選擇訓練、測試欄位

In [31]:
select_column = ['GPM','OPM','NIM','EBITDA','cash','account_receviable','inventory','fixed_asset','invest','liq_asset','liq_liability','leverage','revenue','mkt_value','GOR','EV','EBITDA_EV','change']

x_train = train[select_column]
y_train = train['label']
x_test = test[select_column]
y_test = test['label']

### ．使用隨機森林預測

In [32]:
# 建立模組
rfc = RandomForestClassifier() 
# 訓練模組
rfc.fit(x_train, y_train)
# 預測
EBITDA_EV_predict = rfc.predict(x_test)
EBITDA_EV_predict

array([0, 0, 0, 0, 0, 0, 0, 0])

### ．績效

In [33]:
# 績效
accuracy = metrics.accuracy_score(y_test, EBITDA_EV_predict)
fpr, tpr, thresholds = metrics.roc_curve(y_test, EBITDA_EV_predict)
auc = metrics.auc(fpr, tpr)
#print(auc)
print('準確率: {}'.format(auc))
print('AUC值: {}'.format(accuracy))

準確率: 0.5
AUC值: 0.5


### ．紀錄結果

In [34]:
# EBITDA_EV_predict
apple = pd.DataFrame(EBITDA_EV_predict)

In [35]:
result1 = pd.DataFrame()  
result1 = result1.append(apple)

In [36]:
# code、AUC
result1['code'] = code_list.iloc[1,0]
result1['AUC'] = accuracy

選擇特定列數 :  （.loc[[0,4],:]）

只選擇2016、2017第一季之預測資料

In [37]:
result1 = result1.loc[[0,4],:]

將個股預測資訊，統整為新資料

In [38]:
result = result.append(result1)

In [39]:
result

,0,code,AUC
0,0,1101,1.0
4,0,1101,1.0
0,0,1102,0.5
4,0,1102,0.5


## 四、【迴圈 --- 以code做groupby,　做預測】

建立 code 清單

In [40]:
code_list=pd.DataFrame(data['code'].unique())
code_list.columns=['code']

建立空資料（DataFrame）以記錄迴圈後資料

In [41]:
result = pd.DataFrame()    

In [ ]:
for ix in range(0,code_list.size):
    #創立新的空資料、code清單。
    result1 = pd.DataFrame()  
    pre_data = data[data['code']==code_list.iloc[ix,0]]
    
    # 建立指標欄位
    pre_data['EBITDA_EV_lag'] = pre_data['EBITDA_EV'].shift(+1)
    pre_data['change'] = pre_data['EBITDA_EV']-pre_data['EBITDA_EV_lag']
    pre_data = pre_data.dropna()
    
    pre_data['label'] = np.where(pre_data['change']>=30,1,0)
    
    # train、test
    train = pre_data[pre_data['date']<=201512]
    test = pre_data[pre_data['date']>201512]
    
    select_column = ['GPM','OPM','NIM','EBITDA','cash','account_receviable','inventory','fixed_asset','invest','liq_asset','liq_liability','leverage','revenue','mkt_value','GOR','EV','EBITDA_EV','change']
    x_train = train[select_column]
    y_train = train['label']
    x_test = test[select_column]
    y_test = test['label']
    
    # 使用隨機森林預測
    rfc = RandomForestClassifier() 
    rfc.fit(x_train, y_train)
    EBITDA_EV_predict = rfc.predict(x_test)

    # 績效
    accuracy = metrics.accuracy_score(y_test, EBITDA_EV_predict)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, EBITDA_EV_predict)
    auc = metrics.auc(fpr, tpr)
    
    #紀錄結果
    # EBITDA_EV_predict
    apple = pd.DataFrame(EBITDA_EV_predict)
    result1 = result1.append(apple)
    # code、AUC
    result1['code'] = code_list.iloc[ix,0]
    result1['AUC'] = accuracy
    result1 = result1.loc[[0,4],:]
    
    #合併資料
    result = result.append(result1)
        

更改欄名

In [43]:
result.columns = ['label','code','AUC']

In [44]:
result.head(10)

,label,code,AUC
0,0,1101,0.875
4,0,1101,0.875
0,0,1102,0.875
4,0,1102,0.875
0,0,1103,1.000
4,0,1103,1.000
0,0,1104,1.000
4,0,1104,1.000
0,0,1108,1.000
4,0,1108,1.000


以年份 (2016 、2017)、季拆表 

In [45]:
# 選2016、2017第一季
pre_2016 = result.loc[[0],:]
pre_2017 = result.loc[[4],:]

In [46]:
pre_2016.head()

,label,code,AUC
0,0,1101,0.875
0,0,1102,0.875
0,0,1103,1.000
0,0,1104,1.000
0,0,1108,1.000


依照 EVIT/EV 增加比例（label欄位）和 準確率（AUC），做排序。 

.sort_values(['要降冪排序的欄位'],ascending=False) 

In [47]:
pre_2016 = pre_2016.sort_values(['label','AUC'],ascending=False)
pre_2017 = pre_2016.sort_values(['label','AUC'],ascending=False)

In [48]:
pre_2016.head()

,label,code,AUC
0,1,1432,1.0
0,1,1472,1.0
0,1,2009,1.0
0,1,2364,1.0
0,1,2405,1.0


In [49]:
pre_2016.to_csv('pre_2016.csv')
pre_2017.to_csv('pre_2017.csv')